In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [2]:
from callbacks import get_callbacks

Using TensorFlow backend.


In [3]:
from keras.applications import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.models import load_model
from tensorboard import notebook

In [4]:
from keras import models
from keras import layers
from keras import optimizers

In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils

In [104]:
def save_class_indices(generator, filename):
    class_indices_opp = {v:k for k, v in generator.class_indices.items()}
    
    def save_obj(obj, name ):
        with open(name + '.pkl', 'wb') as f:
            pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
    save_obj(class_indices_opp, filename)
    return generator.class_indices, class_indices_opp

In [7]:
def get_training_directories(base_dir):
    train_dir = os.path.join(base_dir, 'train')
    validation_dir = os.path.join(base_dir, 'val')
    test_dir = os.path.join(base_dir, 'test')
    return train_dir, validation_dir, test_dir

In [8]:
base_dir = 'data/sets/categories_castle_cafe_categories_2_photos'
no_cats = 42

In [9]:
BATCH_SIZE = 2

In [10]:
def get_cnn_data_generators():
    train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
    
    test_datagen = ImageDataGenerator(rescale=1./255)
    return train_datagen, test_datagen
    

In [11]:
def get_image_batches(train_datagen, test_datagen, batch_size):
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

    validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)
    
    # TODO: Test generator
    return train_generator, validation_generator

In [12]:
def get_conv_base(cnn):
    if cnn == 'vgg16':
        conv_base = VGG16(weights='imagenet',
                           include_top=False,
                           input_shape=(150, 150, 3))
    elif cnn == 'inception':
        conv_base = InceptionV3(weights='imagenet',
                                 include_top=False,
                                 input_shape=(150, 150, 3))
    elif cnn == 'xception':
        conv_base = Xception(weights='imagenet',
                             include_top=False,
                             input_shape=(150, 150, 3))
    else:
        raise ValueError(f'Unknown pre-trained CNN. Got {cnn} whereas vgg16, inception or exception is expected.')
    
    return conv_base

Complete the first 3 steps:

    1) Add your custom network on top of an already trained base network.
    2) Freeze the base network.
    3) Train the part you added.

In [13]:
def get_cnn_model(pretrained_cnn, no_cats):
    conv_base = get_conv_base(pretrained_cnn)
    conv_base.trainable = False
    
    model = models.Sequential()
    model.add(conv_base)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(Dense(no_cats, activation='softmax'))

    print(model.summary())
    print(len(model.trainable_weights))
    return model

In [14]:
def build_compile_cnn(cnn, no_cats):
    model = get_cnn_model(cnn, no_cats)
    model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
                  loss='categorical_crossentropy',
                  metrics=['acc'])
    return model

In [15]:
train_dir, validation_dir, test_dir = get_training_directories(base_dir)

train_datagen, test_datagen = get_cnn_data_generators()

train_generator, validation_generator = get_image_batches(train_datagen, test_datagen, BATCH_SIZE)

val_class_indicies, val_class_indicies_opp = save_class_indices(validation_generator, 'class_indices_cafe')

model = build_compile_cnn('vgg16', no_cats)

callbacks = get_callbacks('vgg16_first_go', model, train_generator, validation_generator)

history = model.fit_generator(train_generator,
                              steps_per_epoch=train_generator.samples/train_generator.batch_size,
                              epochs=300,
                              verbose=1,
                              validation_data=validation_generator,
                              callbacks=callbacks
                             )

Found 42 images belonging to 42 classes.
Found 42 images belonging to 42 classes.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 42)                10794     
Total params: 16,822,890
Trainable params: 2,108,202
Non-trainable params: 14,714,688
_________________________________________________________________
None
4


/usr/local/lib/python3.6/site-packages/keras/callbacks/tensorboard_v2.py:92: UserWarning: The TensorBoard callback `batch_size` argument (for histogram computation) is deprecated with TensorFlow 2.0. It will be ignored.
  warnings.warn('The TensorBoard callback `batch_size` argument '
/usr/local/lib/python3.6/site-packages/keras/callbacks/tensorboard_v2.py:97: UserWarning: The TensorBoard callback does not support gradients display when using TensorFlow 2.0. The `write_grads` argument is ignored.
  warnings.warn('The TensorBoard callback does not support '


Epoch 1/300
21/21 [==============================] - 10s 468ms/step - loss: 4.1558 - acc: 0.0238 - val_loss: 4.0539 - val_acc: 0.0000e+00
Epoch 2/300
21/21 [==============================] - 7s 334ms/step - loss: 4.0713 - acc: 0.0238 - val_loss: 4.0294 - val_acc: 0.0000e+00
Epoch 3/300
21/21 [==============================] - 11s 542ms/step - loss: 4.0194 - acc: 0.0238 - val_loss: 3.9822 - val_acc: 0.0000e+00

Epoch 00003: val_loss improved from inf to 3.98220, saving model to callbacks/models/vgg16_first_go_2019-12-08_12-01-07.hdf5
Epoch 4/300
21/21 [==============================] - 9s 430ms/step - loss: 3.9259 - acc: 0.0000e+00 - val_loss: 3.9543 - val_acc: 0.0000e+00
Epoch 5/300
21/21 [==============================] - 10s 497ms/step - loss: 4.0028 - acc: 0.0238 - val_loss: 4.0324 - val_acc: 0.0000e+00
Epoch 6/300
21/21 [==============================] - 7s 354ms/step - loss: 3.7695 - acc: 0.0714 - val_loss: 3.9677 - val_acc: 0.0000e+00

Epoch 00006: val_loss improved from 3.98220 

In [16]:
history = model.fit_generator(train_generator,
                              steps_per_epoch=train_generator.samples/train_generator.batch_size,
                              epochs=300,
                              verbose=1,
                              validation_data=validation_generator,
                              callbacks=callbacks
                             )

Epoch 1/300
21/21 [==============================] - 11s 526ms/step - loss: 0.5733 - acc: 0.8571 - val_loss: 2.6295 - val_acc: 0.4524
Epoch 2/300
21/21 [==============================] - 7s 334ms/step - loss: 0.4846 - acc: 0.9524 - val_loss: 2.5615 - val_acc: 0.4286
Epoch 3/300
21/21 [==============================] - 9s 422ms/step - loss: 0.4618 - acc: 0.9286 - val_loss: 2.5276 - val_acc: 0.4762

Epoch 00003: val_loss improved from 2.54267 to 2.52762, saving model to callbacks/models/vgg16_first_go_2019-12-08_12-01-07.hdf5
Epoch 4/300
21/21 [==============================] - 7s 344ms/step - loss: 0.4441 - acc: 1.0000 - val_loss: 2.5115 - val_acc: 0.4286
Epoch 5/300
21/21 [==============================] - 9s 415ms/step - loss: 0.6241 - acc: 0.8810 - val_loss: 2.4617 - val_acc: 0.4048
Epoch 6/300
21/21 [==============================] - 6s 292ms/step - loss: 0.4613 - acc: 0.9048 - val_loss: 2.4877 - val_acc: 0.4286

Epoch 00006: val_loss improved from 2.52762 to 2.48767, saving model t

KeyboardInterrupt: 

Steps: 4-5

    4) Unfreeze some layers in the base network.
    5) Jointly train both these layers and the part you added.

In [ ]:
model = load_model('castle_30_vgg_features_with_data_augmentation.h5')

In [ ]:
len(model.trainable_weights)

If loaded in 

In [ ]:
# set_trainable = False
# for layer in model.get_layer("vgg16")._layers:
#     layer.trainable = False
#     if layer.name == 'block5_conv1':
#         set_trainable = True
#     if set_trainable:
#         layer.trainable = True
#     else:
#         layer.trainable = False

In [ ]:
# len(model.trainable_weights)

If using conv_base:

In [ ]:
conv_base.trainable = True

In [ ]:
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
len(model.trainable_weights)

In [ ]:
model.compile(optimizer=optimizers.RMSprop(lr=1e-5),
              loss='categorical_crossentropy',
              metrics=['acc'])

In [ ]:
history = model.fit_generator(train_generator,
                    epochs=70,
                    validation_data=validation_generator,
#                     callbacks=[es]
                             )

#### Plotting predictions

In [105]:
from file_processing import listdir_no_hidden, get_filenames

In [106]:
np.set_printoptions(suppress=True)

In [107]:
CATEGORY_PATH = 'categories_castle_cafe/categories_2_photos'

In [108]:
list_of_cats = listdir_no_hidden(os.path.join('data', CATEGORY_PATH))

In [109]:
def get_probs(model, validation_generator):
    pred_probs = model.predict_generator(validation_generator)
    preds = {'first': [], 'second': []}
    for i in pred_probs:
        indices = i.argsort()[-3:][::-1]
        preds['first'].append(indices[0])
        preds['second'].append(indices[1])
    return preds

In [116]:
def get_pred_cat(cat, rank, preds):
    category_index_model = val_class_indices[cat]
    index_in_labels = labels.index(category_index_model)
    pred = preds[rank][index_in_labels]
    pred_category = val_class_indices_opp[pred]
    return pred_category

In [117]:
def get_photo(category, index, group):
    category_set_path = os.path.join("data/sets", CATEGORY_PATH.replace("/", "_"), group, category)
    img_path = os.path.join(category_set_path, get_filenames(category_set_path)[index])
    img = load_img(img_path)      
    return img

In [118]:
def plot_results(list_of_cats):
    list_of_cats = sorted(list_of_cats, key=int)
    preds = get_probs(model, validation_generator)
    fig, ax = plt.subplots(len(list_of_cats), 3, figsize=(15,80))
    for i, c in enumerate(list_of_cats):
        ax[i, 0].imshow(get_photo(c, 0, 'train'))
        ax[i, 1].imshow(get_photo((get_pred_cat(c, 'first', preds)), 0, 'val'))
        ax[i, 2].imshow(get_photo((get_pred_cat(c, 'second', preds)), 0, 'val'))

In [ ]:
plot_results(list_of_cats)